<a href="https://colab.research.google.com/github/nan-park/cp1_project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv